In [144]:
import pandas as pd
import numpy as np
from datetime import timedelta

### 1. ITEMID 선정

In [222]:
# Lab Events 가져오기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/LABEVENTS.csv")

In [223]:
# result 1

# LABEVENTS에서 가장 빈도가 높은 상위 30개 ITEMID만 사용
# common_id = df['ITEMID'].value_counts().index[:30].to_list()

# LABEVENTS의 모든 ITEMID 사용
common_id = df['ITEMID'].value_counts().index.to_list()

df1 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_LABITEMS.csv")
df2 = pd.merge(df1, pd.DataFrame(common_id, columns=["ITEMID"]), on="ITEMID")
result1 = df2[["ITEMID", "LABEL", "FLUID", "CATEGORY", "LOINC_CODE"]]
result1

,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3
1,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
2,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2
3,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6
4,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN
...,...,...,...,...,...
721,51534,MYELOS,JOINT FLUID,HEMATOLOGY,NaN
722,51535,CD55,OTHER BODY FLUID,HEMATOLOGY,NaN
723,51536,CD59,OTHER BODY FLUID,HEMATOLOGY,NaN
724,51537,TDT,OTHER BODY FLUID,HEMATOLOGY,NaN


In [224]:
# result 2

# ARDS 참고 논문의 12개 Feature extract
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_LABITEMS.csv")
L_list = ["WBC Count", "WBC", "Creatinine, Serum", "Albumin", "Platelet Count", "pH", "Lactate", "Oxygen Saturation", "Oxygen", "pCO2", "heart rate", "Temperature"]
result2 = df[df['LABEL'].isin(L_list)]
result2

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
140,14,50813,Lactate,Blood,Blood Gas,32693-4
143,17,50816,Oxygen,Blood,Blood Gas,19994-3
144,18,50817,Oxygen Saturation,Blood,Blood Gas,20564-1
145,19,50818,pCO2,Blood,Blood Gas,11557-6
147,21,50820,pH,Blood,Blood Gas,11558-4
152,26,50825,Temperature,Blood,Blood Gas,NaN
158,32,50831,pH,Other Body Fluid,Blood Gas,2748-2
189,63,50862,Albumin,Blood,Chemistry,1751-7
407,281,51081,"Creatinine, Serum",Urine,Chemistry,NaN
420,294,51094,pH,Urine,Chemistry,2756-5


In [225]:
# result1&result2 결합
result = pd.concat([result1, result2], axis=0)
r_list = result["ITEMID"].tolist()
print("결합 후:",len(r_list))

# 겹치는 값 제거
result = result.drop_duplicates(subset=['ITEMID'])

# ITEMID 리스트에 담기
i_list = result["ITEMID"].tolist()
print("제거 후:",len(i_list))
result

결합 후: 740
제거 후: 726


,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE,ROW_ID
0,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3,NaN
1,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5,NaN
2,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2,NaN
3,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6,NaN
4,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN,NaN
...,...,...,...,...,...,...
721,51534,MYELOS,JOINT FLUID,HEMATOLOGY,NaN,NaN
722,51535,CD55,OTHER BODY FLUID,HEMATOLOGY,NaN,NaN
723,51536,CD59,OTHER BODY FLUID,HEMATOLOGY,NaN,NaN
724,51537,TDT,OTHER BODY FLUID,HEMATOLOGY,NaN,NaN


### 2. Data 가져오기

In [226]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/LABEVENTS.csv")

In [227]:
# 선정된 ITEMID와 겹치는 데이터 추출
df1 = df[df['ITEMID'].isin(r_list)]

# patient table(호흡기 질환 진단 받은 환자)에서 겹치는 데이터만 남기기 위해 HADM_ID int로 변경
df1['HADM_ID'] = df1['HADM_ID'].fillna(0).astype(int)
df1

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,0,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,0,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,0,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,0,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,0,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal
...,...,...,...,...,...,...,...,...,...
27854050,27428435,96443,103219,50882,2109-12-30 01:40:00,26,26.00,mEq/L,NaN
27854051,27428436,96443,103219,50885,2109-12-30 01:40:00,2.1,2.10,mg/dL,abnormal
27854052,27428437,96443,103219,50902,2109-12-30 01:40:00,97,97.00,mEq/L,NaN
27854053,27428438,96443,103219,50911,2109-12-30 01:40:00,2,2.00,ng/mL,NaN


In [228]:
# ARDS 환자 ID와 겹치는 데이터만 남기기
patients = pd.read_csv("~/project/MIMIC-III/Data/OI&OSI/new_OI&OSI(down).csv")
patients = patients[["SUBJECT_ID", "HADM_ID", "y"]]

# 조인하여 데이터 추출
merged = pd.merge(df1, patients, on=['SUBJECT_ID', 'HADM_ID'], how='inner')

# 필요한 칼럼만 선택 + SUBJECT_ID 기준 정렬
df3 = merged[['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUE', 'VALUENUM']]
df3 = df3.sort_values(by='SUBJECT_ID', ascending=True)
df3

,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM
0,101,175533,50868,2196-09-26 12:47:00,28,28.00
416,101,175533,50820,2196-09-29 13:50:00,7.42,7.42
417,101,175533,50821,2196-09-29 13:50:00,82,82.00
418,101,175533,50822,2196-09-29 13:50:00,3.4,3.40
419,101,175533,50826,2196-09-29 13:50:00,300,300.00
...,...,...,...,...,...,...
1946961,32800,179038,50998,2189-02-09 18:22:00,133,133.00
1946962,32800,179038,51283,2189-02-09 18:22:00,2.1,2.10
1946963,32800,179038,50868,2189-02-10 05:30:00,10,10.00
1946949,32800,179038,51249,2189-02-09 06:06:00,33.2,33.20


In [229]:
# 환자별 첫 24시간 이내의 데이터만 남기기
df3['CHARTTIME'] = pd.to_datetime(df3['CHARTTIME']) # CHARTTIME 컬럼을 datetime 자료형으로 변환

df4 = df3.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False).apply(lambda x: x[(x['CHARTTIME'] == x['CHARTTIME'].min()) | ((x['CHARTTIME'] > x['CHARTTIME'].min()) & (x['CHARTTIME'] <= x['CHARTTIME'].min() + pd.Timedelta(hours=24)))])
df4 = df4.reset_index(drop=True)
df4

,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM
0,101,175533,50868,2196-09-26 12:47:00,28,28.00
1,101,175533,51256,2196-09-26 14:05:00,75.7,75.70
2,101,175533,51265,2196-09-26 14:05:00,229,229.00
3,101,175533,51274,2196-09-26 14:05:00,13.1,13.10
4,101,175533,51275,2196-09-26 14:05:00,37.5,37.50
...,...,...,...,...,...,...
351800,32800,179038,50820,2189-02-02 17:58:00,6.99,6.99
351801,32800,179038,50821,2189-02-02 17:58:00,338,338.00
351802,32800,179038,50825,2189-02-02 17:58:00,36.3,36.30
351803,32800,179038,50882,2189-02-02 19:15:00,LESS THAN 5,NaN


###  3. Mean, Min, Max + Missing Value

In [230]:
################ Mean, Min, Max 설정 ################

# 환자별 첫 24시간 이내의 데이터 구하기
df4.fillna(0, inplace=True)  # 결측치를 0으로 변경
df_mean = df4.groupby(['SUBJECT_ID', 'HADM_ID', 'ITEMID'], as_index=False).agg({'VALUENUM': lambda x: np.min(x.astype(float))})
df_mean

,SUBJECT_ID,HADM_ID,ITEMID,VALUENUM
0,101,175533,50800,0.00
1,101,175533,50801,622.00
2,101,175533,50802,-12.00
3,101,175533,50804,13.00
4,101,175533,50808,1.04
...,...,...,...,...
169004,32800,179038,51508,0.00
169005,32800,179038,51514,0.00
169006,32800,179038,51516,0.00
169007,32800,179038,51519,0.00


In [231]:
# 최초 laboratory value의 pivot table 생성
pivot_table = df_mean.pivot_table(index=['SUBJECT_ID', 'HADM_ID'], columns='ITEMID', values='VALUENUM', aggfunc='first')
pivot_table = pivot_table.fillna(0)
df_pivot = pivot_table.reset_index(level=['SUBJECT_ID', 'HADM_ID'])
df_pivot

ITEMID,SUBJECT_ID,HADM_ID,50800,50801,50802,50803,50804,50805,50806,50808,...,51512,51513,51514,51515,51516,51517,51518,51519,51520,51523
0,101,175533,0.0,622.0,-12.0,0.0,13.0,0.0,0.0,1.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,111,192123,0.0,0.0,-5.0,0.0,23.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
2,141,168006,0.0,350.0,-13.0,0.0,12.0,0.0,0.0,0.00,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,148,199488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
4,170,175350,0.0,0.0,2.0,0.0,27.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,32763,160695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2278,32779,133757,0.0,0.0,-8.0,0.0,22.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2279,32781,142684,0.0,269.0,-12.0,0.0,14.0,0.0,0.0,0.94,...,0.0,0.0,4.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
2280,32786,165351,0.0,0.0,-4.0,0.0,21.0,0.0,106.0,1.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [232]:
# 환자별 laboratory value 추가하기
patients = patients.sort_values(by='SUBJECT_ID', ascending=True)
patients = pd.merge(patients[["SUBJECT_ID", "HADM_ID"]], df_pivot, on=['SUBJECT_ID', 'HADM_ID'], how='left')
patients

,SUBJECT_ID,HADM_ID,50800,50801,50802,50803,50804,50805,50806,50808,...,51512,51513,51514,51515,51516,51517,51518,51519,51520,51523
0,101,175533,0.0,622.0,-12.0,0.0,13.0,0.0,0.0,1.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,111,192123,0.0,0.0,-5.0,0.0,23.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
2,141,168006,0.0,350.0,-13.0,0.0,12.0,0.0,0.0,0.00,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,148,199488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
4,170,175350,0.0,0.0,2.0,0.0,27.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,32763,160695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2278,32779,133757,0.0,0.0,-8.0,0.0,22.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2279,32781,142684,0.0,269.0,-12.0,0.0,14.0,0.0,0.0,0.94,...,0.0,0.0,4.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
2280,32786,165351,0.0,0.0,-4.0,0.0,21.0,0.0,106.0,1.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [233]:
# 칼럼 별로 관측되지 않은 laboratory value 값(0) 비율 확인 
zero_ratios = (patients == 0).sum() / len(patients)
zero_ratios = zero_ratios.to_frame()[2:]
zero_ratios = zero_ratios.reset_index().rename(columns={'index': 'ITEMID', 0:'ZERO_RATIOS'})
zero_ratios = pd.merge(zero_ratios, result[["ITEMID", "LABEL", "FLUID"]], on='ITEMID')
zero_ratios = zero_ratios.reindex(columns=["ITEMID", "LABEL", "FLUID", "ZERO_RATIOS"])
zero_ratios = zero_ratios.sort_values(by='ZERO_RATIOS', ascending=True)
zero_ratios

,ITEMID,LABEL,FLUID,ZERO_RATIOS
87,50912,Creatinine,Blood,0.002191
160,51006,Urea Nitrogen,Blood,0.002629
288,51221,Hematocrit,Blood,0.003506
77,50902,Chloride,Blood,0.004382
323,51265,Platelet Count,Blood,0.004382
...,...,...,...,...
284,51217,Glyco A,Blood,1.000000
286,51219,H/O Smear,Blood,1.000000
287,51220,Heinz Body Prep,Blood,1.000000
266,51187,CD55,Blood,1.000000


In [245]:
################ Missing Value 설정 ################


# 관측되지 않은 값이 n% 미만인 ITEMID만 선정
data = zero_ratios[zero_ratios['ZERO_RATIOS'] <= 0.01]
below_list = data['ITEMID'].tolist()
print("ITEMID 개수:", len(below_list))
# data

ITEMID 개수: 17


In [246]:
# mean_patients에서 관측되지 않은 값이 n%이하인 칼럼의 값만 가져오기
i_list = ['SUBJECT_ID', 'HADM_ID']
i_list.extend(data['ITEMID'].tolist())

result = patients[i_list]

# 칼럼 명 변경을 위해 dict 생성 후 rename
id_dict = dict(zip(data['ITEMID'], data['LABEL']))
result = result.rename(columns=id_dict)
result

,SUBJECT_ID,HADM_ID,Creatinine,Urea Nitrogen,Hematocrit,Chloride,Platelet Count,Sodium,White Blood Cells,Glucose,Hemoglobin,MCHC,Anion Gap,Bicarbonate,Red Blood Cells,RDW,MCH,MCV,Potassium
0,101,175533,0.8,15.0,33.4,105.0,199.0,140.0,10.7,186.0,11.0,32.4,16.0,12.0,3.43,11.8,31.9,98.0,3.5
1,111,192123,0.6,12.0,37.4,104.0,131.0,140.0,9.1,144.0,12.8,34.3,10.0,22.0,4.09,12.4,30.6,89.0,4.0
2,141,168006,1.3,26.0,16.9,97.0,51.0,139.0,0.7,111.0,5.0,27.5,17.0,14.0,2.26,13.0,21.9,75.0,3.4
3,148,199488,0.8,16.0,23.9,105.0,20.0,140.0,10.7,87.0,7.8,32.7,15.0,20.0,2.57,18.6,30.0,92.0,3.7
4,170,175350,0.7,13.0,31.2,107.0,238.0,142.0,11.1,101.0,10.7,34.0,13.0,24.0,3.64,13.2,29.3,86.0,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,32763,160695,1.4,35.0,30.2,98.0,181.0,133.0,7.6,136.0,10.8,35.7,16.0,24.0,3.53,14.5,30.5,86.0,4.5
2278,32779,133757,1.4,29.0,26.1,112.0,199.0,143.0,12.0,129.0,8.8,32.1,14.0,18.0,2.96,14.4,29.1,90.0,4.3
2279,32781,142684,1.9,26.0,23.8,105.0,77.0,139.0,3.0,90.0,7.7,32.0,13.0,12.0,2.65,21.2,29.0,90.0,3.3
2280,32786,165351,1.0,13.0,35.6,99.0,311.0,137.0,19.7,136.0,12.4,33.9,13.0,21.0,3.88,13.2,31.2,89.0,4.2


In [247]:
result.to_csv("~/project/MIMIC-III/Data/FINAL/New_Lab/LABORATORY_VALUE_24h(1%)_min.csv", index=False)